## **Análise e Pré-Processamento dos dados Encontrados**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

### **Dando uma olhada nos dados que temos**
Como parte de nossos dados não vem de um Kaggle mas sim de um repositório de webscraping, não tem muita documentação sobre ele, só um artigo não muito claro

OBS: Para não ter chance de perder os dados e também para ter uma visualização melhor, coloquei parte deles num Banco de Dados no postgresSQL

In [ ]:
df_womens_indv = pd.read_csv('C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/volleyball-case-study/data/vnl-2021-2023/womens/df_womens_indv_21_23.csv')
df_womens_indv.head(5)

In [ ]:
df_womens_rosters = pd.read_csv('C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/volleyball-case-study/data/vnl-2021-2023/womens/df_womens_rosters_21_23.csv')
df_womens_rosters.head(5)

In [ ]:
ncaa1 = pd.read_csv("C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/volleyball-case-study/data/fivb-ncaa-2019/ncaa/combined/accumulated/0.2_ewm_combined.csv")
ncaa1.head(5)

In [ ]:
ncaa2 = pd.read_csv("C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/volleyball-case-study/data/fivb-ncaa-2019/ncaa/combined/accumulated/10_sma_combined.csv")
ncaa2.head(5)

In [ ]:
ncaa3= pd.read_csv("C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/volleyball-case-study/data/fivb-ncaa-2019/ncaa/combined/accumulated/cma_combined.csv")
ncaa3.head()

In [ ]:
ncaa4 = pd.read_csv("C:/Users/gabri/OneDrive/Documents/Faculdade/2024.2/Projeto de Ciência de Dados/volleyball-case-study/data/fivb-ncaa-2019/ncaa/combined/accumulated/team_v_team.csv")
ncaa4.head()

### **Dataset escolhido para tratar por enquanto: ``team_v_team.csv``**

* Consiste basicamente de partidas de 2016 até 2019 entre times que são colocados nas posições de time A e time B, talvez haja repretição de dados nas colunas A e B

* Essa feature 'results' ta meio simplificada, em um outro csv tem essa 'results' com o valor de quantos sets ganhos 

**Explicação das colunas:**

Fonte das Explicações: https://wiaawi.org/Portals/0/PDF/Sports/Volleyball/vbstatspresentation.pdf#:~:text=What%20is%20hitting%20percentage?%20*%20Answer:%20(kills,Note%20that%20if%20a%20player%20has%20more

* **S**: quantos Sets tiveram no total

* **Team # Kills**: Quantos ataques não defendidos pelo time oposto, que diretamente resultam em ponto

* **Team # Errors**: Ataque que diretamente resulta em ponto para o time oposto, bola na rede por exemplo

* **Team # Total Attacks**: Total de ataques dos jogadores (kills-errors)/total

* **Team # Hit Pct**: (kills-errors)/total_attacks

* **Team # Assists**: Quando um jogador passe, prepara ou ""mancheteia" a bola para um parceiro de equipe que consegue uma Kill a partir disso

* **Team # Aces**: Um "Service Ace" é dado quando -> um saque que atinge a quadra intocada, que é tocada mas não continua no jogo, em que é dado violação ao time recebedor ou no caso do time recebedor estar fora de rotação

* **Team # SErr**: Um "Service Error" é dado quando -> um saque que não passa da rede, passa dos limites da quadra, pé do sacador fora de posição ou quando o time que está sacando está fora de rotação

* **Team # Digs**: Famosa manchete

* **Team # RErr**: Uma "Reception Error" é dada quando o time oposto marca uma pontuação, ela é atribuida a um jogador do time

* **Team # Block Solos**: bloqueio de um único jogador que diretamente leva a um ponto

* **Team # Block Assists**: quando a tentativa de bloqueio de mais de um jogador leva a um ponto

* **Team # BErr**: Quando a tentativa de bloqueio de uma bola diretamente resulta na parade de uma jogada e na pontuação do time oposto

* **Team # PTS**: Kills + Aces + Block Solos + 1/2 Block Assists


### **Verificando se existem dados ""invertidos"" no dataframe**
Existem registros duplicados da mesma partida apenas invertendo quem está como "Team A" e "Team B". Porém como esses dados foram tirados via webscraping e devem ter páginas diferentes para cada time como "Team A" existe uma inconsistência nos dados, o que pode trazer problemas no futuro **melhor retirá-los**.

In [6]:
caso_1=ncaa4[(ncaa4['TeamA']== 'USC Upstate') & (ncaa4['TeamB']=='Ga. Southern')]
caso_2 = ncaa4[(ncaa4['TeamB']== 'USC Upstate') & (ncaa4['TeamA']=='Ga. Southern')]

In [25]:
caso_1

,Unnamed: 0,Date,TeamA,TeamB,Result,S,Team A Kills,Team A Errors,Team A Total Attacks,Team A Hit Pct,...,Team B Hit Pct,Team B Assists,Team B Aces,Team B SErr,Team B Digs,Team B RErr,Team B Block Solos,Team B Block Assists,Team B BErr,Team B PTS
2,2,9072019,USC Upstate,Ga. Southern,0,3.0,37,20,117,0.145,...,0.239,39,4,5,49,2,1,6,1,50.0
9544,9544,9142018,USC Upstate,Ga. Southern,0,4.0,46,34,170,0.071,...,0.192,49,2,7,80,1,2,35,1,80.5


In [23]:
caso_2

,Unnamed: 0,Date,TeamA,TeamB,Result,S,Team A Kills,Team A Errors,Team A Total Attacks,Team A Hit Pct,...,Team B Hit Pct,Team B Assists,Team B Aces,Team B SErr,Team B Digs,Team B RErr,Team B Block Solos,Team B Block Assists,Team B BErr,Team B PTS
7144,7144,9072019,Ga. Southern,USC Upstate,1,3.0,42,15,113,0.239,...,0.145,35,2,4,48,4,4,2,2,44.0
16482,16482,9142018,Ga. Southern,USC Upstate,1,4.0,59,27,167,0.192,...,0.071,43,1,7,65,2,1,22,2,59.0


In [ ]:
caso_2['Team A Kills']

In [ ]:
caso_1['Team B Kills']